In [1]:
import torch
import torch.nn as nn
import torch.jit

In [3]:
def graph_inputs(graph):
    """Returns inputs of graph as list of torch._C.Value"""
    return list(graph.inputs())

def graph_outputs(graph):
    """Returns outputs of graph as list of torch._C.Value"""
    return list(graph.outputs())

def val_size(val):
    return val.type().sizes()

def val_inputs(val):
    return list(val.node().inputs())

def val_outputs(val):
    return list(val.node().outputs())

def val_kind(val):
    return val.node().kind()

def val_is_tensor(val):
    return val.isCompleteTensor()

def val_scope_name(val):
    return val.node().scopeName()

def val_tensor(val):
    return val.node().t('value')

In [4]:
class MyModule(nn.Module):
    
    def __init__(self):
        super(MyModule, self).__init__()
        self.a = torch.randn(1, 3, 1, 1)
        
    def forward(self, x):
        return x * torch.ones(1) * self.a

In [79]:
model = MyModule()

data = torch.zeros((1, 3, 224, 224))

model_jit = torch.jit.trace(model, [data])

graph = model_jit.graph

In [159]:
queue = []

queue += graph_outputs(graph)

while queue:
    val = queue.pop(0)
    queue += val_inputs(val)
    print(val)
    
    if val_kind(val) == 'prim::Constant':
        break

11 defined in (%11 : Float(1, 3, 224, 224) = aten::mul(%9, %10), scope: MyModule # <ipython-input-78-2efbdc103968>:8:0
)
9 defined in (%9 : Float(1, 3, 224, 224) = aten::mul(%x, %8), scope: MyModule # <ipython-input-78-2efbdc103968>:8:0
)
10 defined in (%10 : Float(1, 3, 1, 1) = prim::Constant[value=(1,1,.,.) =   -0.5971  (1,2,.,.) =    0.3635  (1,3,.,.) =    0.4893 [ Variable[CPUFloatType]{1,3,1,1} ]](), scope: MyModule # <ipython-input-78-2efbdc103968>:8:0
)
